# Big Data & BI — Feature Engineering
## Notebook 2: Imputation & Standardization

Now we **apply** the cleaning plan from notebook 1:
- parse dates with mixed formats
- standardize country/channel labels
- impute missing numeric fields
- create cleaned columns we can rely on later

In [ ]:
import pandas as pd
import numpy as np

data = {
    "order_id":   [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010],
    "order_date": ["2025-01-03", "2025/01/03", "03-01-2025", "2025-01-04", None,
                    "2025-01-05", "2025-01-05", "2025-01-06", "2025-01-06", "2025-01-06"],
    "customer_id": [501, 502, 503, 503, 504, 505, 506, 506, 507, None],
    "country":    ["DE", "Germany", "germany", "FR", "France", "DE", "DE ", "?", None, "GER"],
    "product":    ["Widget A", "Widget B", "Widget A", "Widget C", "Widget A",
                    "Widget B", "Widget B", "Widget C", "Widget A", "Widget A"],
    "quantity":   [2, 1, 3, 1, -1, 2, 2, 1, 5, 2],
    "unit_price": [20.0, 35.5, 20.0, 50.0, 20.0, None, 35.5, 50.0, 20.0, 20.0],
    "discount":   [0.0, 0.1, None, 0.0, 0.0, 0.05, 0.0, None, 0.0, 0.0],
    "channel":    ["online", "Online", "offline", "partner", "online",
                    "offline", "online ", "ONLINE", None, "partner"]
}
df = pd.DataFrame(data)

## 1. Parse messy dates → datetime
We allow errors and convert them to NaT, we'll decide later whether to drop or impute.

In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce")
df[["order_id", "order_date"]]

## 2. Standardize text columns
We do lowercase + strip and then map weird values to a standard form.

**Two approaches:**
1. **Keep both columns** (what we do here) → safer for auditing, you can compare before/after
2. **Replace directly** → saves memory, cleaner final dataframe

We keep both so you can verify the transformations worked correctly. In the final preview (section 5), we'll drop the original messy columns.

In [ ]:
# country
# Step 1: Convert to lowercase and remove leading/trailing spaces
df["country_clean"] = df["country"].str.lower().str.strip()

# Step 2: Define a mapping dictionary (key = messy value, value = standard form)
country_map = {
    "de": "germany",
    "germany": "germany",
    "ger": "germany",
    "fr": "france",
    "france": "france",
    "?": "unknown",
    None: "unknown"
}

# Step 3: Apply the mapping
# .map(country_map) → replaces values according to the dictionary
# .fillna(df["country_clean"]) → if a value is NOT in the map, keep the original cleaned value
df["country_clean"] = df["country_clean"].map(country_map).fillna(df["country_clean"])

# channel
# Step 1: Convert to lowercase and remove leading/trailing spaces
df["channel_clean"] = df["channel"].str.lower().str.strip()

# Step 2: Define a mapping dictionary
channel_map = {
    "online": "online",
    "offline": "offline",
    "partner": "partner",
    "": "unknown",
    None: "unknown"
}

# Step 3: Apply the mapping
# .map(channel_map) → replaces values according to the dictionary
# .fillna(df["channel_clean"]) → if a value is NOT in the map, keep the original cleaned value
df["channel_clean"] = df["channel_clean"].map(channel_map).fillna(df["channel_clean"])

# Display the results: show original and cleaned versions side-by-side
# .drop_duplicates() → remove duplicate rows to see unique transformations only
# .reset_index(drop=True) → renumber rows from 0, 1, 2...
df[["country", "country_clean", "channel", "channel_clean"]].drop_duplicates().reset_index(drop=True)

Let's now have another look at the dataframe:

In [ ]:
df

---

## What is Imputation?

**Imputation** = filling in missing values with estimated/reasonable values instead of deleting rows.

### Why do we need imputation?

In real-world data, missing values are common:
- Sensors fail → missing temperature readings
- Users skip survey questions → missing demographic data
- System errors → missing prices or dates

### Two main approaches:

#### 1. **Delete rows with missing values** (simple but wasteful)
- Pro: Clean data, no assumptions
- Con: Lose potentially valuable information
- Con: Can create bias (what if missing values aren't random?)
- **Use when**: Very few missing values (<5%) and they're truly random

#### 2. **Impute (fill) missing values** (more sophisticated)
- Pro: Keep all rows, preserve data volume
- Pro: Can maintain statistical properties
- Con: Introduces assumptions/estimates
- **Use when**: Missing values are common or systematic

### Common Imputation Strategies:

| Strategy | How it works | When to use | Example |
|----------|-------------|-------------|---------|
| **Constant** | Fill with a fixed value | Categorical data, safe defaults | Fill missing country with "unknown" |
| **Mean** | Fill with average of all values | Numerical, normally distributed | Average temperature |
| **Median** | Fill with middle value | Numerical, with outliers | Prices (robust to extreme values) |
| **Mode** | Fill with most common value | Categorical data | Most common product category |
| **Forward/Backward fill** | Use previous/next value | Time series data | Stock prices |
| **Group-specific** | Fill based on category | When patterns differ by group | Price by product type |
| **Model-based** | Predict using ML | Complex relationships | Predict based on multiple features |

### For our BI use case:
- **Numeric fields** (price, discount) → median or mean (median is safer with outliers)
- **Categorical fields** (country, channel) → constant like "unknown" or mode
- **IDs** (customer_id) → single placeholder value like -1 or "UNKNOWN" or unique placeholder value if our values might be used as a primary key or identification way.
- **Dates** → depends on context (might need to delete if critical for time series)


## Note
**Important**: Always document what imputation you did! Your stakeholders need to know which values are real vs. estimated.

## 3. Impute numeric columns
- price → median (simple, robust)
- discount → 0.0 (safe default in BI)
- quantity → we'll fix negative in next notebook, so we keep it for now

**Why median for missing price?**
- We have ONE missing price (row with order_id 1006, Widget B)
- We could look up "Widget B costs 35.5" BUT that only works if we know the product
- **Median is a general strategy** that works even when we don't know the product or don't have a price lookup table
- In real BI pipelines, you often can't manually check each missing value → automated imputation is needed

In [ ]:
# Let's see which row has missing price
print("Row with missing price:")
print(df[df["unit_price"].isna()][["order_id", "product", "unit_price"]])

print("\nAll prices by product:")
print(df.groupby("product")["unit_price"].describe())

print(f"\nMedian of ALL prices: {df['unit_price'].median()}")
print("\nSo we'll fill the missing Widget B price with 20.0 (the median)")
print("A BETTER approach would be to use product-specific prices, but median is simpler and more general.")

In [ ]:
median_price = df["unit_price"].median()
df["unit_price_filled"] = df["unit_price"].fillna(median_price)
df["discount_filled"] = df["discount"].fillna(0.0)
df[["unit_price", "unit_price_filled", "discount", "discount_filled"]]

## 4. Customer IDs

**Problem**: We have missing customer IDs, but customer_id might be a primary key!

**Three approaches:**

1. **Single placeholder** (e.g., -1 or 0) → Simple but loses uniqueness
   - Can't use as primary key anymore
   - Can't distinguish between different unknown customers
   
2. **Unique negative IDs** (e.g., -1, -2, -3...) → Preserves uniqueness
   - Can still use as primary key
   - Negative IDs can cause issues in SQL (foreign key constraints, WHERE clauses, confusion)
   - May break business logic that assumes IDs > 0

3. **Unique IDs outside normal range** (e.g., 999000001, 999000002...) → Best practice
   - Can still use as primary key
   - No negative number issues in SQL/BI tools
   - Easy to identify as placeholder (start with 999...)
   - Won't conflict with real customer IDs (assuming they're < 999000000)

We'll use **unique high-value IDs (999000001+)** - the safest approach for production systems.

In [ ]:
# Option 1: Single placeholder (BAD for primary keys)
# df["customer_id_clean"] = df["customer_id"].fillna(-1).astype(int)

# Option 2: Unique negative IDs (OK but can cause SQL issues)
# missing_mask = df["customer_id"].isna()
# num_missing = missing_mask.sum()
# df["customer_id_clean"] = df["customer_id"].copy()
# df.loc[missing_mask, "customer_id_clean"] = range(-1, -1 - num_missing, -1)

# Option 3: Unique high-value IDs (BEST for production)
# Dynamically find the safe range based on actual data
missing_mask = df["customer_id"].isna()
num_missing = missing_mask.sum()

# Find the maximum existing customer ID to avoid conflicts
max_existing_id = df["customer_id"].max()
print(f"Maximum existing customer_id: {max_existing_id}")

# Choose a placeholder start that's clearly outside the normal range
# Strategy: round up to next order of magnitude and add safety margin
# E.g., if max is 507, use 10000 or 100000
import math
if pd.notna(max_existing_id):
    # Calculate order of magnitude and go to next higher one
    order_of_magnitude = 10 ** (math.floor(math.log10(max_existing_id)) + 2)
    placeholder_start = order_of_magnitude
else:
    # If all IDs are missing, start from a safe default
    placeholder_start = 999000001

print(f"Placeholder IDs will start from: {placeholder_start}")

# Generate unique placeholder IDs
df["customer_id_clean"] = df["customer_id"].copy()
df.loc[missing_mask, "customer_id_clean"] = range(placeholder_start, placeholder_start + num_missing)
df["customer_id_clean"] = df["customer_id_clean"].astype(int)

# Verify: check if customer_id_clean is unique (can be used as primary key)
print(f"\nTotal rows: {len(df)}")
print(f"Unique customer_id_clean values: {df['customer_id_clean'].nunique()}")
print(f"Is unique (can be primary key)? {df['customer_id_clean'].is_unique}")
print(f"Placeholder IDs used: {sorted(df.loc[missing_mask, 'customer_id_clean'].values)}")

df[["customer_id", "customer_id_clean"]]

## 5. Preview cleaned frame
This is what we'll feed into the next notebook for integrity checks.

In [ ]:
clean_cols = [
    "order_id", "order_date", "customer_id_clean", "country_clean",
    "product", "quantity", "unit_price_filled", "discount_filled", "channel_clean"
]
df_clean = df[clean_cols].copy()
df_clean